# Diagrama de Classes


```mermaid

classDiagram
    namespace Domain_Model {
        class User {
            -Long id
            -String name
            -String email
            -String password
            -String salt
            -LocalDateTime createdAt
            -LocalDateTime updatedAt
            -UserStatus status
            -List~UserEmail~ emails
            -List~UserAddress~ addresses
            -List~UserTelephone~ telephones
            +to(UserCreateDTO data, String password, String salt)$ User
            +getEmails() Collection~UserEmail~
            +addEmail(String email, UserEmailType type, boolean isPrimary)
            +getAddresses() Collection~UserAddress~
            +addAddress(UserAddressType type, ..., CEP zipCode, Boolean isPrimary)
            +getTelephones() Collection~UserTelephone~
            +addTelephone(TelephoneType type, String areaCode, String telephone, Boolean isPrimary)
            +setPrimaryTelephone(Long telephoneId)
        }

        class UserEmail {
            -Long id
            -User user
            -UserEmailType type
            -String email
            -Boolean isPrimary
            -EmailStatus status
            +removePrimary()
        }

        class UserAddress {
            -Long id
            -User user
            -UserAddressType type
            -String street
            -String number
            -String city
            -CEP zipCode
            -Boolean isPrimary
            -UserAddressStatus status
            +removePrimary()
        }

        class UserTelephone {
            -Long id
            -User user
            -TelephoneType type
            -String areaCode
            -String telephone
            -Boolean isPrimary
            -UserTelephoneStatus status
            +removePrimary()
            +setPrimary()
        }
    }

    namespace DTOs {
        class UserCreateDTO {
            <<Record>>
            +String name
            +String email
            +String password
            +String salt
        }
    }

    namespace Application_Layer {
        class UserController {
            -UserAssembler assembler
            -UserQuery query
            -UserCommand command
            +findAll(Pageable pageable) PagedModel
            +findById(Long id) ResponseEntity
            +saveUser(UserCreateDTO data) ResponseEntity
        }

        class UserCommand {
            -UserRepository repository
            +create(UserCreateDTO data) User
        }

        class UserQuery {
            <<Interface>>
            +findById(Long id) User
            +findAll(Pageable pageable) Page~User~
        }

        class UserRepository {
            <<Interface>>
            +existsByEmail(String email) boolean
        }
    }

%% Relacionamentos
    User "1" *-- "*" UserEmail : has
    User "1" *-- "*" UserAddress : has
    User "1" *-- "*" UserTelephone : has

    UserController ..> UserQuery : uses
    UserController ..> UserCommand : uses
    UserController ..> UserCreateDTO : receives

    UserCommand ..> UserRepository : uses
    UserCommand ..> User : creates
    UserCommand ..> UserCreateDTO : reads

    UserQuery ..> User : returns
    UserRepository --|> JpaRepository : extends
    UserRepository ..> User : manages

```

# Diagrama de Sequência

```mermaid

sequenceDiagram
    autonumber
    actor Client as Usuário/Frontend

    box Application Layer
        participant Controller as CepLookupController
        participant Orchestrator as AddressLookupOrchestrator
    end

    box Infrastructure / Adapter Layer
        participant Provider as ViaCepProvider
        participant FeignClient as ViaCepClient
    end

    box External System
        participant ExternalAPI as API ViaCep
    end

    Client->>Controller: GET /v1/finance/utils/cep/{cep}
    activate Controller

    Note right of Client: Inicia busca de endereço via CEP

    Controller->>Orchestrator: getAddressBy(cep)
    activate Orchestrator

    Orchestrator->>Provider: getAddressBy(cep)
    activate Provider

    Provider->>FeignClient: getAddress(cep)
    activate FeignClient

    FeignClient->>ExternalAPI: HTTP GET [https://viacep.com.br/](https://viacep.com.br/)...
    activate ExternalAPI
    ExternalAPI-->>FeignClient: JSON Response (ViaCepResponse)
    deactivate ExternalAPI

    FeignClient-->>Provider: ViaCepResponse
    deactivate FeignClient

    Note right of Provider: Converte ViaCepResponse para AddressInfoDTO

    Provider-->>Orchestrator: AddressInfoDTO
    deactivate Provider

    Orchestrator-->>Controller: AddressInfoDTO
    deactivate Orchestrator

    Controller-->>Client: 200 OK (AddressInfoDTO)
    deactivate Controller

```

# Diagrama Entidade-Relacionamento


```mermaid

erDiagram
    user ||--o{ user_email : "possui"
    user ||--o{ user_address : "reside em"
    user ||--o{ user_telephone : "contatado via"

    user {
        bigint id PK
        varchar name
        varchar email UK "Unique Key"
        varchar password
        varchar salt
        varchar status "Enum: ACTIVE, etc"
        timestamp created_at
        timestamp updated_at
    }

    user_email {
        bigint id PK
        bigint user_id FK
        varchar email
        varchar type "Enum: PERSONAL, WORK"
        boolean is_primary
        varchar status
    }

    user_address {
        bigint id PK
        bigint user_id FK
        varchar street
        varchar number
        varchar city
        varchar state
        varchar zip_code
        boolean is_primary
        varchar country "Default: BRA"
        varchar status
    }

    user_telephone {
        bigint id PK
        bigint user_id FK
        varchar area_code
        varchar telephone
        varchar type "Enum: MOBILE, LANDLINE"
        boolean is_primary
        varchar status
    }

```

# Criação de um usuário

``` mermaid

sequenceDiagram
    autonumber
    actor Client as Cliente/Frontend
    participant Controller as UserController
    participant Command as UserCommand
    participant Repo as UserRepository
    participant DB as Banco de Dados

    Client->>Controller: POST /v1/finance/user (UserCreateDTO)
    activate Controller

    Controller->>Command: create(UserCreateDTO)
    activate Command

    Note right of Command: 1. Validação de Regra de Negócio

    Command->>Repo: existsByEmail(email)
    activate Repo
    Repo-->>Command: boolean exists
    deactivate Repo

    alt Email já existe
        Command-->>Controller: Throws IllegalArgumentException
        Controller-->>Client: 400 Bad Request ("Email já cadastrado")
    else Email disponível
        Note right of Command: 2. Preparação dos dados
        Command->>Command: Gerar Salt e Hash Senha

        Command->>Repo: save(User)
        activate Repo
        Repo->>DB: <<CREATE>>
        DB-->>Repo: User persistido (com ID)
        Repo-->>Command: User
        deactivate Repo

        Command-->>Controller: User criado

        Controller-->>Client: 201 Created (User Response)
    end

    deactivate Command
    deactivate Controller

```

## Criação de um email para um usuário existente

``` mermaid

sequenceDiagram
    autonumber
    actor Client as Usuário/Frontend
    participant API as UserEmailController
    participant Cmd as UserEmailCommand
    participant Repo as UserRepository
    participant User as User (Aggregate)

    Client->>API: POST /v1/finance/user/{userId}/email
    activate API

    API->>Cmd: create(userId, UserEmailCreateDTO)
    activate Cmd

    Cmd->>Repo: findById(userId)
    activate Repo
    Repo-->>Cmd: User (Optional)
    deactivate Repo

    opt Usuário não encontrado
        Cmd-->>API: Throws Exception
        API-->>Client: 404 Not Found
    end

    Cmd->>User: addEmail(email, type, isPrimary)
    activate User

    alt Email já vinculado
        User-->>Cmd: Throws IllegalArgumentException
    else Novo Email
        Note right of User: 1. Ajusta flags isPrimary se necessário<br/>2. Adiciona à lista interna
        User-->>Cmd: void
    end
    deactivate User

    Cmd->>Repo: save(User)
    activate Repo
    Repo-->>Cmd: User salvo
    deactivate Repo

    Note right of Cmd: Recupera o objeto UserEmail criado da lista

    Cmd-->>API: UserEmail (Saved)
    deactivate Cmd

    API-->>Client: 201 Created
    deactivate API

```

# Criação de Telefone

```mermaid
sequenceDiagram
    autonumber
    actor Client
    participant API as UserTelephoneController
    participant Cmd as UserTelephoneCommand
    participant Repo as UserRepository
    participant User as User (Aggregate)

    Client->>API: POST .../user-telephones
    activate API
    
    API->>Cmd: create(request, userId)
    activate Cmd
    
    Cmd->>Repo: findById(userId)
    Repo-->>Cmd: User
    
    Cmd->>User: addTelephone(type, areaCode, tel, isPrimary)
    activate User
    
    Note right of User: 1. Valida Unicidade (Tipo + DDD + Número)
    alt Já existe
        User-->>Cmd: Throws Exception ("Telephone.already.exists")
        Cmd-->>API: 400 Bad Request
    else Novo Telefone
        Note right of User: 2. Calcula isPrimary
        opt isPrimaryInput OR Primeiro do Tipo
             User->>User: unsetPrimaryTelephonesByType(type)
        end
        User->>User: add(new UserTelephone)
    end
    deactivate User

    Cmd->>Repo: save(User)
    
    Note right of Cmd: Busca na lista (Tipo + DDD + Número)

    Cmd-->>API: UserTelephone
    deactivate Cmd

    API-->>Client: 201 Created
    deactivate API
```

# Criação de Endereço

``` mermaid

sequenceDiagram
    autonumber
    actor Client
    participant API as UserAddressController
    participant Cmd as UserAddressCommand
    participant Repo as UserRepository
    participant User as User (Aggregate)
    participant CEP as CEP (Value Object)

    Client->>API: POST .../address
    activate API

    API->>Cmd: create(userId, DTO)
    activate Cmd

    Cmd->>Repo: findById(userId)
    Repo-->>Cmd: User

    Note right of Cmd: Cria objeto CEP com status 'UNVERIFIED'
    Cmd->>CEP: unverified(zipCodeString)
    CEP-->>Cmd: CEP Object

    Cmd->>User: addAddress(..., zipCode, isPrimary)
    activate User
    Note right of User: Adiciona endereço à coleção
    deactivate User

    Cmd->>Repo: save(User)

    Note right of Cmd: Recupera o endereço mais recente (max createdAt)

    Cmd-->>API: UserAddress
    deactivate Cmd

    API-->>Client: 201 Created
    deactivate API

```

# Ciclo de Vida e Navegação

### Diagrama de Estados (User Status)
```mermaid
stateDiagram-v2
    [*] --> ACTIVE : Cadastro Realizado
    
    ACTIVE --> SUSPENDED : Violação Leve
    ACTIVE --> BLOCKED : Violação Grave / Fraude
    ACTIVE --> INACTIVE : Solicitação do Usuário
    
    SUSPENDED --> ACTIVE : Resolução de Pendência
    SUSPENDED --> BANNED : Reincidência
    
    BLOCKED --> BANNED : Análise Final
    BLOCKED --> ACTIVE : Desbloqueio Administrativo
    
    INACTIVE --> ACTIVE : Reativação
    
    BANNED --> [*]
```

### Mapa de Navegação da API (Mindmap)
```mermaid
%%{init: { 'theme': 'forest' } }%%
mindmap
  root((Finance API))
    v1/finance/user
      GET / (Listar Usuários)
      POST / (Criar Usuário)
      GET /id/:id (Buscar por ID)
      ::icon(fa fa-user)

      /:userId/address
        GET / (Listar Endereços)
        POST / (Adicionar Endereço)
        ::icon(fa fa-home)

      /:userId/user-telephones
        GET / (Listar Telefones)
        POST / (Adicionar Telefone)
        PATCH /:telephoneId/primary (Definir como Principal)
        ::icon(fa fa-phone)

      /:userId/email
        GET / (Listar Emails)
        POST / (Adicionar Email)
        ::icon(fa fa-envelope)

```